In [2]:
import numpy as np
import tensorflow as tf

In [3]:
class Preprocessing:
    
    def __init__(self,nr_of_classes=4,image_shape =(32, 32, 3)):
        self.mean=np.zeros([3])
        self.std=np.zeros([3]) 
        self.nr_of_classes=nr_of_classes
        self.nomalize_data=True
        self.image_shape=image_shape
        self.x = tf.placeholder(tf.float32, shape=(None,)+self.image_shape)
        self.y = tf.placeholder(tf.int32)
        self.Y = tf.placeholder(tf.float32, shape=(None,self.nr_of_classes))
        self.x_rot =  tf.image.rot90(self.x, self.y)
        

    def set_params(self,nr_of_classes=None,input_shape=None,norm_params=None,img=None):
        if nr_of_classes!=None:    self.nr_of_classes=nr_of_classes                 
        if input_shape!=None:      self.input_shape=input_shape
        if norm_params!=None and img!=None:
            img_array = np.asarray(img)
            for i in range(3):
                self.mean[i]=img_array[:,:,:,i].mean()
                self.std[i]=img_array[:,:,:,i].std()
        elif norm_params!=None and img==None:
              print("Data is required")


    def normalize(self,img,img2):
        img_array = np.asarray(img)
        img_array2 = np.asarray(img2)
        normalized = np.zeros(np.shape(img_array))
        normalized2 = np.zeros(np.shape(img_array2))
        if self.nomalize_data:
            for i in range(3):
                self.mean[i]=img_array[:,:,:,i].mean()
                self.std[i]=img_array[:,:,:,i].std()
        for i in range(3):
            normalized = (img_array - self.mean[i]) / (self.std[i])
            normalized2 = (img_array2 - self.mean[i]) / (self.std[i])
        return normalized,normalized2
    

    def get_one_hot_rot(self,x,K):
        one_hot=np.zeros((np.shape(x)[0],K))
        one_hot[:,0]=1
        for j in range(K-1):
            labels=np.zeros((np.shape(x)[0],K))
            labels[:,j+1]=1
            one_hot=np.concatenate((one_hot,labels))
        return one_hot
    
    
    def get_one_hot(self,y,K):
        one_hot=np.zeros((np.shape(y)[0],K))
        for j in range(np.shape(y)[0]):
            one_hot[j,y[j]]=1
        return one_hot
    
    
    def create_rotated_data(self,x_train,x_test,nomalize_data=False):
        if self.nomalize_data:
            x_train,x_test=self.normalize(x_train,x_test)
            self.nomalize_data=False
                      
        elif nomalize_data==True:
            x_train,x_test=self.normalize(x_train,x_test)
            
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess: 
            sess.run(tf.global_variables_initializer())
            labels=self.get_one_hot_rot(x_train,self.nr_of_classes)
            x_train_rot=sess.run ( self.x_rot, feed_dict={self.x:x_train,self.y:0})
            for j in range(self.nr_of_classes-1):
                x_train_rot=np.concatenate((x_train_rot,sess.run ( self.x_rot, feed_dict={self.x:x_train, self.y:(j+1)})))
            x_test_rot=sess.run ( self.x_rot, feed_dict={self.x:x_test,self.y:0})
            labels_test=self.get_one_hot_rot(x_test,self.nr_of_classes)
            for j in range(self.nr_of_classes-1):
                x_test_rot=np.concatenate((x_test_rot,sess.run ( self.x_rot, feed_dict={self.x:x_test, self.y:(j+1)})))                
                
        return [x_train_rot,labels,x_test_rot,labels_test]
      
      
      
    def create_data(self,x_train,y_train,x_test,y_test,nomalize_data=False):
        if self.nomalize_data:
            x_train,x_test=self.normalize(x_train,x_test)
            self.nomalize_data=False

        elif nomalize_data==True:
            x_train,x_test=self.normalize(x_train,x_test)

        labels=self.get_one_hot(y_train,self.nr_of_classes)
        labels_test=self.get_one_hot(y_test,self.nr_of_classes)
        return [x_train,labels,x_test,labels_test] 

In [4]:
class NetWorkInNetwork:
    
    def __init__(self,model=None,nr_of_classes=4,nr_of_blocks=4,DECAY = 5e-4 ,input_shape=(32,32,3),conv__layer_strides=[1,1],pooling_layer_strides=(2,2)):
        if model==None:
            self.model=tf.keras.models.Sequential()
            self.cov_1_created=False
        else:
            self.model=model
            self.cov_1_created=True

        self.nr_of_classes=nr_of_classes
        self.nr_of_blocks=nr_of_blocks
        self.DECAY = DECAY
        self.conv_strides=conv__layer_strides
        self.pooling_strides=pooling_layer_strides
        self.input_shape=input_shape
        
        
    def set_params(nr_of_classes=None,nr_of_blocks=None,DECAY = None ,input_shape=None,conv__layer_strides=None,pooling_layer_strides=None):
        
        if nr_of_classes!=None:    self.nr_of_classes=nr_of_classes        
        if nr_of_blocks!=None:     self.nr_of_blocks=nr_of_blocks      
        if DECAY!=None:            self.DECAY = DECAY       
        if input_shape!=None:      self.input_shape=input_shape
        if conv_strides!=None:     self.conv_strides=conv_strides        
        if pooling_strides!=None:  self.pooling_strides = pooling_strides
        
        
    def conv_layer(self,kernel_size,out_channels):
        
        my_init=tf.keras.initializers.RandomNormal(mean=0.0, stddev=tf.math.sqrt(2/(kernel_size*kernel_size*out_channels)))
        if not self.cov_1_created:
            self.cov_1_created=True
            self.model.add(tf.keras.layers.Conv2D(out_channels, kernel_size=kernel_size,strides=self.conv_strides,
                                                  kernel_initializer=my_init,
                                                  kernel_regularizer=tf.keras.regularizers.l2(self.DECAY), 
                                                  use_bias=False ,
                                                  input_shape=self.input_shape,
                                                  padding='same',
                                                  data_format='channels_last'))         
        else:
            self.model.add(tf.keras.layers.Conv2D(out_channels, kernel_size=kernel_size,strides=self.conv_strides,
                                                  kernel_initializer=my_init,
                                                  kernel_regularizer=tf.keras.regularizers.l2(self.DECAY),
                                                  use_bias=False ,
                                                  padding='same'))
        
        self.model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05))
        self.model.add(tf.keras.layers.ReLU())
                           
                           
        
    def create_block(self,block_type="middle",kernel_list=[3,1,1],channels_list=[192,192,192]):
                           
        self.conv_layer(kernel_list[0],channels_list[0])
        self.conv_layer(kernel_list[1],channels_list[1])
        self.conv_layer(kernel_list[2],channels_list[2])
                   
        if block_type=="first":
                self.model.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
                self.model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=self.pooling_strides))
                           
                           
        elif block_type=="last":
            self.model.add(tf.keras.layers.GlobalAveragePooling2D())
            my_init_dense=tf.keras.initializers.VarianceScaling(scale=1.0, mode='fan_in', distribution='uniform')
            self.model.add(tf.keras.layers.Dense(self.nr_of_classes, activation=tf.nn.softmax,
                                                 kernel_initializer=my_init_dense,
                                                 kernel_regularizer=tf.keras.regularizers.l2(self.DECAY),
                                                 bias_initializer='zeros',
                                                 bias_regularizer=tf.keras.regularizers.l2(self.DECAY)))
                        
        else:
            self.model.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
            self.model.add(tf.keras.layers.AveragePooling2D(pool_size=(3, 3), strides=self.pooling_strides))
                           
                           
    def create_network(self,nr_of_blocks=None,return_model=False):
        if nr_of_blocks==None:
            nr_of_blocks=self.nr_of_blocks
    
        self.create_block(block_type="first",kernel_list=[5,1,1],channels_list=[192,160,96])
        self.create_block(kernel_list=[5,1,1])
        if nr_of_blocks>2:
            for i in range(3,nr_of_blocks):
                self.create_block() 
        self.create_block(block_type="last")
        if return_model:
            return self.model
        
    def train_model(self):
        my_optimizer=tf.keras.optimizers.SGD(momentum=MOMENTUM, nesterov=True)
        self.model.compile(optimizer=my_optimizer,loss=cross_entropy_loss,metrics=['accuracy'])
        self.model.fit(batch_x_rot,labels, batch_size=512, epochs=100, callbacks=callbacks_list)
        
    def get_model(self):
        return self.model

In [5]:
class Train_and_Evaluate:
    def __init__(self, model, epochs=100, batch_size=128, Momentum=0.9, Decay=5e-4,initial_lrate=0.1,lr_decay=5,limit_1=30,limit_2=60,limit_3=80,limit_4=None):
        self.model = model
        self.epochs = epochs
        self.batch_size = batch_size
        self.Momentum = Momentum
        self.Losshistory=self.LossHistory(self)
        self.initial_lrate=float(initial_lrate)
        self.lr_decay=lr_decay
        self.limit_1=limit_1
        self.limit_2=limit_2
        self.limit_3=limit_3
        if limit_4!=None:
            self.limit_4=limit_4
        else:
            self.limit_4=epochs+1
        
    def cross_entropy_loss(self, F, Y):
        return -tf.math.reduce_mean(tf.math.log(tf.reduce_sum(F * Y, 1)), 0)

    def step_decay(self, epoch):
        if epoch <=self.limit_1:
            lrate = self.initial_lrate
        elif epoch <=self.limit_2:
            lrate = self.initial_lrate / self.lr_decay
        elif epoch <=self.limit_3:
            lrate = self.initial_lrate / pow(self.lr_decay,2)
        elif epoch <=self.limit_4:
            lrate = self.initial_lrate / pow(self.lr_decay,3)   
        else:
            lrate = self.initial_lrate / pow(self.lr_decay,4)
        return lrate
    
    def set_step_decay(self,initial_lrate=None, lr_decay=None,limit_1=None,limit_2=None,limit_3=None,limit_4=None):
        if initial_lrate!=None: self.initial_lrate=float(initial_lrate)
        if lr_decay!=None: self.lr_decay=lr_decay
        if limit_1!=None:  self.limit_1=limit_1
        if limit_2!=None:  self.limit_2=limit_2
        if limit_3!=None:  self.limit_3=limit_3
        if limit_4!=None:  self.limit_4=limit_4
        
    
    class LossHistory(tf.keras.callbacks.Callback):
    
        def __init__(self,train_and_evaluate):
            self.train_and_evaluate=train_and_evaluate

        def on_train_begin(self, logs={}):
            self.losses = []
            self.lr = []

        def on_epoch_end(self, batch, logs={}):
            self.losses.append(logs.get('loss'))
            self.lr.append(self.train_and_evaluate.step_decay(len(self.losses)))
            print('lr:', self.train_and_evaluate.step_decay(len(self.losses)))

    
    def train(self,x_train,labels):
        loss_history = self.Losshistory
        lrate = tf.keras.callbacks.LearningRateScheduler(self.step_decay)
        callbacks_list = [loss_history, lrate]
        my_optimizer = tf.keras.optimizers.SGD(momentum=self.Momentum, nesterov=True)
        self.model.compile(optimizer=my_optimizer, loss=self.cross_entropy_loss, metrics=['accuracy'])
        self.model.fit(x_train, labels, batch_size=self.batch_size, epochs=self.epochs, callbacks=callbacks_list)

    def evaluate(self, x_test, labels, get_value=False):
        loss, acc = self.model.evaluate(x_test, labels)
        if get_value:
            return loss, acc
        
    def get_model(self):
        return self.model
        
        

In [6]:

class Transfer_learning(Train_and_Evaluate):   

    def __init__(self,model,classifier="nonlinear",blocks=2,freez=True,old_model_blocks=4):
        
        if isinstance(model, str):
            # When we trained our models, we used the name loss_kev for the cross_entropy_loss that we defined.
            # So by using try and except we can load both the models that where trained before and after we
            # changed the name of the loss without retraining them.  
            try:
                self.model = tf.keras.models.load_model(model,custom_objects={'cross_entropy_loss': super().cross_entropy_loss})
            except:
                self.model = tf.keras.models.load_model(model,custom_objects={'loss_kev': super().cross_entropy_loss})
        else:
            self.model=model

        self.classifier=classifier
        self.blocks=blocks
        self.freez=freez
        self.old_model_blocks=old_model_blocks

    def add_nonlinear_classifier(self):
        my_init_dense=tf.keras.initializers.VarianceScaling(scale=2.0, mode='fan_out', distribution='normal')
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(200, activation=None,kernel_initializer=my_init_dense, kernel_regularizer=tf.keras.regularizers.l2(5e-4),use_bias=False))
        self.model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05, name='bn1'))
        self.model.add(tf.keras.layers.ReLU(name='relu1'))
        self.model.add(tf.keras.layers.Dense(200, activation=None,kernel_initializer=my_init_dense, kernel_regularizer=tf.keras.regularizers.l2(5e-4),use_bias=False))
        self.model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05, name='bn2'))
        self.model.add(tf.keras.layers.ReLU(name='relu2'))
        self.model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax,kernel_initializer=my_init_dense, kernel_regularizer=tf.keras.regularizers.l2(5e-4), bias_initializer='zeros',bias_regularizer=tf.keras.regularizers.l2(5e-4)))


    def add_convolutional_classifier(self):
        my_init_4=tf.keras.initializers.RandomNormal(mean=0.0, stddev=np.math.sqrt(2.0/192))
        my_init_5=tf.keras.initializers.RandomNormal(mean=0.0, stddev=np.math.sqrt(2.0/(3*3*192)))
        my_init_dense=tf.keras.initializers.VarianceScaling(scale=2.0, mode='fan_out', distribution='normal')
        self.model.add(tf.keras.layers.Conv2D(192,kernel_size=3,strides=[1,1], kernel_initializer=my_init_5,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same', name='conv1'))
        self.model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05, name='bn1'))
        self.model.add(tf.keras.layers.ReLU(name='relu1'))
        self.model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same', name='conv2'))
        self.model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05, name='bn2'))
        self.model.add(tf.keras.layers.ReLU(name='relu2'))
        self.model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same', name='conv3'))
        self.model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05, name='bn3'))
        self.model.add(tf.keras.layers.ReLU(name='relu3'))
        self.model.add(tf.keras.layers.GlobalAveragePooling2D())
        self.model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax,kernel_initializer=my_init_dense, kernel_regularizer=tf.keras.regularizers.l2(5e-4), bias_initializer='zeros',bias_regularizer=tf.keras.regularizers.l2(5e-4)))

    def transfer_algorithm(self,freez=True):
        self.freez=freez
        add_padding_and_avg_pol=False
        if self.blocks < self.old_model_blocks:
            layer_index = self.blocks * 11
        else:
            layer_index = self.blocks*11-2
            add_padding_and_avg_pol = True
        new_model = tf.keras.models.Sequential()
        for i in range(layer_index):
            layer = self.model.get_layer(index=i)
            if self.freez:
                layer.trainable = False
            new_model.add(layer)
            
        if add_padding_and_avg_pol:
            new_model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05))
            new_model.add(tf.keras.layers.ReLU())

        self.model=new_model
        if self.classifier == "nonlinear":
            self.add_nonlinear_classifier()
        elif self.classifier == "convolutional":
            self.add_convolutional_classifier()
        else:
            print("One of the strings nonlinear or convolutional is required")
        return self.model
    

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
process_data=Preprocessing()
x_train_rot,labels,x_test_rot,labels_test=process_data.create_rotated_data(x_train,x_test)

RotNet=NetWorkInNetwork()
RotNet.create_network()
RotNet=RotNet.get_model()
train_RotNet=Train_and_Evaluate(RotNet, batch_size=512)
print("Training RotNet")
train_RotNet.train(x_train_rot,labels)
print("Evaluating RotNet")
train_RotNet.evaluate(x_train_rot,labels)
RotNet=train_RotNet.get_model()
RotNet.save("NetWorkInNetwork_4_Blocks.model")



print("\n\n")
print("Now we move on to classification of the images themselves")
print("\n\n")
process_data.set_params(nr_of_classes=10)
x_train,labels,x_test,labels_test=process_data.create_data(x_train,y_train,x_test,y_test,nomalize_data=True)

print("Block 1 and non-linear classifier")
B1_non_lin=Transfer_learning(RotNet,blocks=1).transfer_algorithm()
train_B1_non_lin=Train_and_Evaluate(B1_non_lin)
train_B1_non_lin.set_step_decay(limit_1=20,limit_2=40,limit_3=45,limit_4=50)
print("Training B1_non_lin")
train_B1_non_lin.train(x_train,labels)
print("Evaluating B1_non_lin")
train_B1_non_lin.evaluate(x_test,labels_test)
B1_non_lint=train_B1_non_lin.get_model()

print("\n\n")
print("Block 2 and non-linear classifier")
B2_non_lin=Transfer_learning(RotNet,blocks=2).transfer_algorithm()
train_B2_non_lin=Train_and_Evaluate(B2_non_lin)
print("Training B2_non_lin")
train_B2_non_lin.set_step_decay(limit_1=20,limit_2=40,limit_3=45,limit_4=50)
train_B2_non_lin.train(x_train,labels)
print("Evaluating B2_non_lin")
train_B2_non_lin.evaluate(x_test,labels_test)
B2_non_lin=train_B2_non_lin.get_model()

print("\n\n")
print("Block 2 and non-linear classifier")
B3_non_lin=Transfer_learning(RotNet,blocks=3).transfer_algorithm()
train_B3_non_lin=Train_and_Evaluate(B3_non_lin)
print("Training B3_non_lin")
train_B3_non_lin.set_step_decay(limit_1=20,limit_2=40,limit_3=45,limit_4=50)
train_B3_non_lin.train(x_train,labels)
print("Evaluating B3_non_lin")
train_B3_non_lin.evaluate(x_test,labels_test)
B3_non_lin=train_B3_non_lin.get_model()

print("\n\n")
print("Block 2 and non-linear classifier")
B4_non_lin=Transfer_learning(RotNet,blocks=4).transfer_algorithm()
train_B4_non_lin=Train_and_Evaluate(B4_non_lin)
print("Training B4_non_lin")
train_B4_non_lin.set_step_decay(limit_1=20,limit_2=40,limit_3=45,limit_4=50)
train_B4_non_lin.train(x_train,labels)
print("Evaluating B4_non_lin")
train_B4_non_lin.evaluate(x_test,labels_test)
B4_non_lin=train_B4_non_lin.get_model()

print("\n\n")
print("Block 2 and RotNet+conv classifier")
B2_conv=Transfer_learning(RotNet,blocks=2).transfer_algorithm()
train_B2_conv=Train_and_Evaluate(B2_conv)
print("Training B2_conv")
train_B2_conv.set_step_decay(limit_1=35,limit_2=40,limit_3=85)
train_B2_conv.train(x_train,labels)
print("Evaluating B2_conv")
train_B2_conv.evaluate(x_test,labels_test)
B2_conv=train_B2_conv.get_model()